# Module 2: Google ADK Fundamentals

**Duration:** ~40 minutes

**Learning Objectives:**
- Master the core Agent Development Kit (ADK) concepts
- Build agents with custom tools using `FunctionTool`
- Understand LlmAgent configuration and behavior
- Create multi-agent systems with workflow agents
- Work with sessions, state, and memory

---

## 2.1 ADK Architecture Deep Dive

### What is ADK?

The **Agent Development Kit (ADK)** is Google's open-source framework for building AI agents. It's the same framework that powers Google's internal agent systems like Agentspace and Customer Engagement Suite.

### Core Philosophy

ADK is designed around several key principles:

1. **Code-First**: Agents are Python objects, not YAML configs
2. **Composable**: Build complex systems from simple components
3. **Model-Agnostic**: Works with Gemini, Claude, Ollama, and more
4. **Production-Ready**: Direct path to Vertex AI deployment

### ADK Component Hierarchy

```
┌─────────────────────────────────────────────────────────────────┐
│                         Runner                                   │
│  Orchestrates execution, manages sessions, handles I/O          │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                       Agent                              │   │
│  │  The cognitive unit that processes requests              │   │
│  │  ┌──────────────────────────────────────────────────┐   │   │
│  │  │                    Tools                          │   │   │
│  │  │  Functions the agent can call                     │   │   │
│  │  │  • FunctionTool (Python functions)                │   │   │
│  │  │  • McpToolset (MCP servers)                       │   │   │
│  │  │  • AgentTool (other agents)                       │   │   │
│  │  └──────────────────────────────────────────────────┘   │   │
│  │  ┌──────────────────────────────────────────────────┐   │   │
│  │  │                  Sub-Agents                       │   │   │
│  │  │  Child agents for delegation                      │   │   │
│  │  └──────────────────────────────────────────────────┘   │   │
│  └─────────────────────────────────────────────────────────┘   │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │              Session Service & State                     │   │
│  │  Persists conversation history and agent state           │   │
│  └─────────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup: Run this cell first
import os
from getpass import getpass

# Ensure API key is set
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

# Core imports
from google.adk.agents import Agent, LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
import asyncio
from typing import Dict, Any, List

print("✅ Setup complete!")

## 2.2 Building Your First Agent with Tools

### The FunctionTool Pattern

Tools are the primary way agents interact with the outside world. In ADK, you create tools from Python functions using `FunctionTool`.

**Key Requirements:**
1. Functions should have **type hints** (ADK uses these to generate schemas)
2. Functions should have **docstrings** (used as tool descriptions)
3. Functions can be **sync or async**
4. Return values are passed back to the LLM

In [ ]:
# Example 1: Simple Tool Functions

def get_current_weather(city: str, unit: str = "celsius") -> Dict[str, Any]:
    """
    Get the current weather for a specified city.
    
    Args:
        city: The name of the city to get weather for
        unit: Temperature unit - 'celsius' or 'fahrenheit'
    
    Returns:
        Dictionary containing weather information
    """
    # Simulated weather data (in production, call a real API)
    weather_data = {
        "new york": {"temp_c": 22, "condition": "sunny", "humidity": 45},
        "london": {"temp_c": 15, "condition": "cloudy", "humidity": 70},
        "tokyo": {"temp_c": 28, "condition": "humid", "humidity": 80},
        "sydney": {"temp_c": 18, "condition": "windy", "humidity": 55},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        temp = data["temp_c"]
        if unit.lower() == "fahrenheit":
            temp = (temp * 9/5) + 32
        
        return {
            "status": "success",
            "city": city,
            "temperature": temp,
            "unit": unit,
            "condition": data["condition"],
            "humidity": data["humidity"]
        }
    else:
        return {
            "status": "error",
            "message": f"Weather data not available for {city}"
        }


def calculate_tip(bill_amount: float, tip_percentage: float = 18.0) -> Dict[str, float]:
    """
    Calculate the tip amount and total bill.
    
    Args:
        bill_amount: The original bill amount in dollars
        tip_percentage: The tip percentage (default 18%)
    
    Returns:
        Dictionary with tip amount and total
    """
    tip = bill_amount * (tip_percentage / 100)
    total = bill_amount + tip
    
    return {
        "bill_amount": bill_amount,
        "tip_percentage": tip_percentage,
        "tip_amount": round(tip, 2),
        "total": round(total, 2)
    }


print("✅ Tool functions defined!")
print(f"   • get_current_weather: {get_current_weather.__doc__.split(chr(10))[1].strip()}")
print(f"   • calculate_tip: {calculate_tip.__doc__.split(chr(10))[1].strip()}")

In [ ]:
# Create an agent with tools

weather_tip_agent = Agent(
    name="weather_tip_assistant",
    model="gemini-2.0-flash",
    description="An assistant that can check weather and calculate tips",
    instruction="""
    You are a helpful assistant that can:
    1. Check the weather in various cities
    2. Calculate tips for restaurant bills
    
    When asked about weather, always use the get_current_weather tool.
    When asked about tips, always use the calculate_tip tool.
    Be friendly and provide helpful context with your responses.
    """,
    tools=[
        FunctionTool(get_current_weather),
        FunctionTool(calculate_tip)
    ]
)

print(f"✅ Agent '{weather_tip_agent.name}' created with {len(weather_tip_agent.tools)} tools")

In [ ]:
# Helper function to interact with agents

async def chat_with_agent(agent: Agent, message: str, session_id: str = None) -> str:
    """
    Send a message to an agent and get the response.
    
    Args:
        agent: The ADK agent to interact with
        message: The user's message
        session_id: Optional session ID for continuity
    
    Returns:
        The agent's text response
    """
    runner = InMemoryRunner(agent=agent)
    
    # Create or use existing session
    if session_id is None:
        session = await runner.session_service.create_session(
            app_name=agent.name,
            user_id="workshop_user"
        )
        session_id = session.id
    
    # Run the agent
    response_text = ""
    response = await runner.run(
        user_id="workshop_user",
        session_id=session_id,
        new_message=message
    )
    
    # Collect the response
    async for event in response:
        if hasattr(event, 'content') and event.content:
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text
    
    return response_text, session_id

print("✅ Helper function defined!")

In [ ]:
# Test the agent with weather query

response, session = await chat_with_agent(
    weather_tip_agent, 
    "What's the weather like in Tokyo right now?"
)

print("🌤️ Weather Query Test")
print("=" * 50)
print(f"User: What's the weather like in Tokyo right now?")
print(f"\nAgent: {response}")

In [ ]:
# Test the agent with tip calculation

response, _ = await chat_with_agent(
    weather_tip_agent,
    "My dinner bill is $85.50. What should I tip if I want to leave 20%?"
)

print("💰 Tip Calculation Test")
print("=" * 50)
print(f"User: My dinner bill is $85.50. What should I tip if I want to leave 20%?")
print(f"\nAgent: {response}")

## 2.3 Advanced Tool Patterns

### Async Tools

For I/O-bound operations (API calls, database queries), use async functions:

In [ ]:
import httpx

async def fetch_random_fact() -> Dict[str, str]:
    """
    Fetch a random fun fact from the internet.
    
    Returns:
        Dictionary containing a random fact
    """
    try:
        async with httpx.AsyncClient() as client:
            # Using a free facts API
            response = await client.get(
                "https://uselessfacts.jsph.pl/api/v2/facts/random",
                timeout=10.0
            )
            if response.status_code == 200:
                data = response.json()
                return {
                    "status": "success",
                    "fact": data.get("text", "No fact available")
                }
    except Exception as e:
        pass
    
    # Fallback facts if API fails
    import random
    fallback_facts = [
        "Honey never spoils. Archaeologists found 3000-year-old honey in Egyptian tombs that was still edible.",
        "Octopuses have three hearts and blue blood.",
        "A group of flamingos is called a 'flamboyance'.",
        "The shortest war in history lasted 38-45 minutes between Britain and Zanzibar."
    ]
    return {
        "status": "success",
        "fact": random.choice(fallback_facts)
    }

# Test the async function
result = await fetch_random_fact()
print(f"Random fact: {result['fact']}")

### Tools with Complex Return Types

Tools can return structured data that the LLM interprets:

In [ ]:
from datetime import datetime, timedelta
from typing import List

def search_flights(
    origin: str,
    destination: str,
    date: str,
    passengers: int = 1
) -> Dict[str, Any]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Origin airport code (e.g., 'JFK', 'LAX')
        destination: Destination airport code
        date: Travel date in YYYY-MM-DD format
        passengers: Number of passengers (default 1)
    
    Returns:
        Dictionary containing flight options
    """
    # Simulated flight data
    import random
    
    airlines = ["United", "Delta", "American", "JetBlue"]
    flights = []
    
    for i in range(3):
        departure_hour = 6 + i * 4
        flights.append({
            "flight_number": f"{random.choice(['UA', 'DL', 'AA', 'B6'])}{random.randint(100, 999)}",
            "airline": random.choice(airlines),
            "departure": f"{departure_hour:02d}:00",
            "arrival": f"{departure_hour + 3:02d}:30",
            "duration": "3h 30m",
            "price_per_person": random.randint(150, 450),
            "seats_available": random.randint(5, 50)
        })
    
    return {
        "status": "success",
        "origin": origin.upper(),
        "destination": destination.upper(),
        "date": date,
        "passengers": passengers,
        "flights": flights,
        "total_results": len(flights)
    }


def book_flight(
    flight_number: str,
    passenger_names: List[str],
    contact_email: str
) -> Dict[str, Any]:
    """
    Book a flight for specified passengers.
    
    Args:
        flight_number: The flight number to book
        passenger_names: List of passenger full names
        contact_email: Email for booking confirmation
    
    Returns:
        Booking confirmation details
    """
    import random
    import string
    
    confirmation_code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    
    return {
        "status": "success",
        "confirmation_code": confirmation_code,
        "flight_number": flight_number,
        "passengers": passenger_names,
        "email_sent_to": contact_email,
        "message": f"Booking confirmed! Confirmation code: {confirmation_code}"
    }

print("✅ Flight tools defined!")

In [ ]:
# Create a travel agent with flight tools

travel_agent = Agent(
    name="travel_assistant",
    model="gemini-2.0-flash",
    description="A travel assistant that helps with flight searches and bookings",
    instruction="""
    You are a friendly travel assistant specializing in flight bookings.
    
    Your capabilities:
    1. Search for flights between cities
    2. Help users compare flight options
    3. Book flights for passengers
    
    Guidelines:
    - Always search for flights before recommending options
    - Present flight options in a clear, organized way
    - Confirm booking details before finalizing
    - Be helpful and proactive in suggesting alternatives
    """,
    tools=[
        FunctionTool(search_flights),
        FunctionTool(book_flight)
    ]
)

# Test flight search
response, _ = await chat_with_agent(
    travel_agent,
    "I need to fly from New York (JFK) to Los Angeles (LAX) on March 15th, 2025. What flights are available?"
)

print("✈️ Travel Agent Test")
print("=" * 50)
print(f"Agent Response:\n{response}")

## 2.4 Multi-Agent Systems

ADK supports building systems where multiple agents collaborate. There are two main patterns:

### Pattern 1: Hierarchical Agents (Sub-Agents)

A root agent can delegate to specialized sub-agents:

```
        ┌─────────────────┐
        │   Root Agent    │
        │  (Orchestrator) │
        └────────┬────────┘
                 │
     ┌───────────┼───────────┐
     │           │           │
┌────▼────┐ ┌────▼────┐ ┌────▼────┐
│ Weather │ │ Travel  │ │ Finance │
│  Agent  │ │  Agent  │ │  Agent  │
└─────────┘ └─────────┘ └─────────┘
```

### Pattern 2: Workflow Agents

ADK provides workflow agents for deterministic orchestration:

- **SequentialAgent**: Executes agents in order
- **ParallelAgent**: Executes agents concurrently
- **LoopAgent**: Repeats agents until condition met

In [ ]:
# Create specialized sub-agents

# Weather specialist
weather_specialist = Agent(
    name="weather_specialist",
    model="gemini-2.0-flash",
    description="Specialist for weather-related queries. Use when the user asks about weather, temperature, or climate.",
    instruction="You are a weather expert. Provide detailed weather information and advice.",
    tools=[FunctionTool(get_current_weather)]
)

# Calculator specialist  
def calculate(expression: str) -> Dict[str, Any]:
    """
    Safely evaluate a mathematical expression.
    
    Args:
        expression: A mathematical expression like '2 + 2' or '(10 * 5) / 2'
    
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation (only allows math operations)
        allowed_chars = set('0123456789+-*/(). ')
        if not all(c in allowed_chars for c in expression):
            return {"status": "error", "message": "Invalid characters in expression"}
        
        result = eval(expression)
        return {
            "status": "success",
            "expression": expression,
            "result": result
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

math_specialist = Agent(
    name="math_specialist",
    model="gemini-2.0-flash",
    description="Specialist for calculations and math. Use when the user needs mathematical computations.",
    instruction="You are a math expert. Help with calculations and explain your work.",
    tools=[FunctionTool(calculate), FunctionTool(calculate_tip)]
)

print("✅ Specialist agents created!")

In [ ]:
# Create an orchestrator agent that delegates to specialists

orchestrator_agent = Agent(
    name="personal_assistant",
    model="gemini-2.0-flash",
    description="A personal assistant that coordinates with specialists",
    instruction="""
    You are a personal assistant that helps users with various tasks.
    
    You have access to specialist agents:
    - weather_specialist: For weather queries
    - math_specialist: For calculations
    
    Delegate tasks to the appropriate specialist based on the user's request.
    If a request doesn't fit any specialist, answer it yourself.
    """,
    sub_agents=[weather_specialist, math_specialist]
)

print(f"✅ Orchestrator '{orchestrator_agent.name}' created with {len(orchestrator_agent.sub_agents)} sub-agents")

In [ ]:
# Test the orchestrator with different types of queries

test_queries = [
    "What's the weather in London?",
    "Calculate 15% of 230",
    "What's the capital of France?"  # Should answer directly
]

print("🎯 Orchestrator Agent Test")
print("=" * 60)

for query in test_queries:
    response, _ = await chat_with_agent(orchestrator_agent, query)
    print(f"\n📝 Query: {query}")
    print(f"🤖 Response: {response[:200]}..." if len(response) > 200 else f"🤖 Response: {response}")
    print("-" * 60)

## 2.5 Workflow Agents

For deterministic pipelines, use workflow agents:

In [ ]:
from google.adk.agents import SequentialAgent, ParallelAgent

# Create agents for a content pipeline

# Step 1: Research agent
researcher = Agent(
    name="researcher",
    model="gemini-2.0-flash",
    description="Researches and gathers information on a topic",
    instruction="""
    You are a research assistant. When given a topic:
    1. Identify 3-5 key points about the topic
    2. Note any interesting facts or statistics
    3. Format your findings as bullet points
    
    Output your research in a structured format.
    """
)

# Step 2: Writer agent
writer = Agent(
    name="writer",
    model="gemini-2.0-flash",
    description="Writes engaging content based on research",
    instruction="""
    You are a content writer. Take the research provided and:
    1. Write a compelling 2-3 paragraph article
    2. Use engaging language
    3. Include a catchy opening
    
    Transform bullet points into flowing prose.
    """
)

# Step 3: Editor agent
editor = Agent(
    name="editor",
    model="gemini-2.0-flash",
    description="Reviews and polishes content",
    instruction="""
    You are an editor. Review the content and:
    1. Fix any grammar or spelling issues
    2. Improve clarity and flow
    3. Add a title and summary
    
    Output the final polished version.
    """
)

print("✅ Pipeline agents created!")

In [ ]:
# Create a sequential pipeline

content_pipeline = SequentialAgent(
    name="content_pipeline",
    description="A pipeline that researches, writes, and edits content",
    sub_agents=[researcher, writer, editor]
)

print(f"✅ Sequential pipeline created!")
print(f"   Pipeline: {' → '.join([a.name for a in content_pipeline.sub_agents])}")

In [ ]:
# Test the content pipeline

response, _ = await chat_with_agent(
    content_pipeline,
    "Write an article about the benefits of meditation"
)

print("📝 Content Pipeline Output")
print("=" * 60)
print(response)

## 2.6 State and Memory

ADK provides mechanisms for maintaining state across interactions:

### Session State
- Persists within a single conversation session
- Accessible to agents via `context.state`

### Memory
- Long-term storage across sessions
- Useful for user preferences, learned information

In [ ]:
# Example: Stateful agent that remembers user preferences

def set_user_preference(preference_name: str, preference_value: str) -> Dict[str, str]:
    """
    Store a user preference.
    
    Args:
        preference_name: Name of the preference (e.g., 'favorite_city', 'temperature_unit')
        preference_value: Value of the preference
    
    Returns:
        Confirmation of stored preference
    """
    # In a real implementation, this would store to session state
    # For this demo, we'll use a simple dict
    return {
        "status": "success",
        "message": f"Preference '{preference_name}' set to '{preference_value}'"
    }

def get_user_preferences() -> Dict[str, Any]:
    """
    Retrieve all stored user preferences.
    
    Returns:
        Dictionary of user preferences
    """
    # Demo preferences
    return {
        "status": "success",
        "preferences": {
            "favorite_city": "Tokyo",
            "temperature_unit": "celsius",
            "language": "English"
        }
    }

personalized_agent = Agent(
    name="personalized_assistant",
    model="gemini-2.0-flash",
    description="An assistant that remembers user preferences",
    instruction="""
    You are a personalized assistant that remembers user preferences.
    
    - Use get_user_preferences to check stored preferences
    - Use set_user_preference to save new preferences
    - Refer to preferences when making suggestions
    """,
    tools=[
        FunctionTool(set_user_preference),
        FunctionTool(get_user_preferences),
        FunctionTool(get_current_weather)
    ]
)

# Test personalization
response, _ = await chat_with_agent(
    personalized_agent,
    "What's the weather in my favorite city?"
)

print("🎯 Personalized Agent Test")
print("=" * 50)
print(response)

## 2.7 Best Practices Summary

### Tool Design
✅ **Do:**
- Use clear, descriptive function names
- Add comprehensive docstrings
- Include type hints for all parameters
- Return structured dictionaries
- Handle errors gracefully

❌ **Don't:**
- Create overly complex tools (break them up)
- Use ambiguous parameter names
- Return raw exceptions to the LLM
- Include sensitive logic in tools

### Agent Design
✅ **Do:**
- Write clear, specific instructions
- Use sub-agents for complex tasks
- Test agents with edge cases
- Monitor tool usage patterns

❌ **Don't:**
- Overload agents with too many tools (>10)
- Use vague instructions
- Skip error handling
- Ignore context window limits

## 2.8 Exercises

### Exercise 1: Create a Restaurant Finder Agent

Create an agent with tools to:
1. Search for restaurants by cuisine and location
2. Get restaurant details (rating, hours, price range)
3. Make a reservation

In [ ]:
# Exercise 1: Your code here

# def search_restaurants(cuisine: str, location: str) -> Dict[str, Any]:
#     """Search for restaurants"""
#     pass

# def get_restaurant_details(restaurant_id: str) -> Dict[str, Any]:
#     """Get details for a specific restaurant"""
#     pass

# def make_reservation(restaurant_id: str, party_size: int, datetime: str) -> Dict[str, Any]:
#     """Make a reservation"""
#     pass

# restaurant_agent = Agent(...)

### Exercise 2: Build a Multi-Agent Customer Support System

Create a system with:
1. A triage agent that categorizes requests
2. A billing specialist agent
3. A technical support agent
4. An orchestrator that routes requests

In [ ]:
# Exercise 2: Your code here

# billing_agent = Agent(...)
# tech_support_agent = Agent(...)
# customer_support_orchestrator = Agent(..., sub_agents=[...])

---

## Summary

In this module, you learned:

1. **ADK Architecture**: Runners, Agents, Tools, and Sessions
2. **FunctionTool**: Converting Python functions to agent tools
3. **Tool Patterns**: Sync/async tools, complex return types
4. **Multi-Agent Systems**: Sub-agents and delegation
5. **Workflow Agents**: Sequential and parallel pipelines
6. **State Management**: Session state and memory

### Next Steps

Continue to **Module 3: MCP Integration** to learn how to connect your agents to external tools and services using the Model Context Protocol.

---

### Key Code References

```python
# Creating an agent with tools
agent = Agent(
    name="my_agent",
    model="gemini-2.0-flash",
    instruction="...",
    tools=[FunctionTool(my_function)]
)

# Creating multi-agent system
orchestrator = Agent(
    name="orchestrator",
    sub_agents=[agent1, agent2]
)

# Creating a pipeline
pipeline = SequentialAgent(
    name="pipeline",
    sub_agents=[step1, step2, step3]
)
```